Wybieram najlepszy z zapisanych modeli:

In [1]:
import os
import pandas as pd

models_path = 'saved_models'
accuracies = {}

for model_name in os.listdir(models_path):
    table = pd.read_csv(os.path.join(models_path, model_name, 'gridsearch.csv'))
    accuracies[table.loc[table['model name'] == model_name, 'accuracy'].item()] = model_name

model_name = accuracies[max(accuracies.keys())]
print(f'the best model is {model_name} with accuracy {max(accuracies.keys())}')

the best model is model-15-537838 with accuracy 0.4143454038997214


In [2]:
tokenizer_name = 'bardsai/twitter-sentiment-pl-base'
file_path = '2024-emotion-recognition\\test-B\in.tsv'

In [3]:
import torch
import gc

if torch.cuda.is_available():
    gc.collect()
    torch.cuda.empty_cache()
    print(f'GPU {torch.cuda.get_device_name(0)} will be used.')
    device = torch.device("cuda")

GPU NVIDIA GeForce GTX 1660 Ti will be used.


Ładuję model i tokenizer do pipeline, sprawdzam jak wyglądają wyniki.

In [4]:
from transformers import BertForSequenceClassification, HerbertTokenizer, pipeline

tokenizer = HerbertTokenizer.from_pretrained(tokenizer_name)
model = BertForSequenceClassification.from_pretrained(os.path.join(models_path, model_name))

sentence = 'Jedzenie codziennie to samo: mięso lub ryba, zupa jedna do wyboru.'

nlp = pipeline(task='sentiment-analysis',
               model=model,
               tokenizer=tokenizer,
               device=device,
               top_k=11)
result = nlp(sentence)
result

c:\Users\magda\kogni\inzynieria lingwistyczna\zaliczenie\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\magda\kogni\inzynieria lingwistyczna\zaliczenie\.venv\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


[[{'label': 'Disgust', 'score': 0.9894058108329773},
  {'label': 'Negative', 'score': 0.9815894961357117},
  {'label': 'Sadness', 'score': 0.9735413789749146},
  {'label': 'Joy', 'score': 0.2635478675365448},
  {'label': 'Positive', 'score': 0.16346825659275055},
  {'label': 'Surprise', 'score': 0.07592661678791046},
  {'label': 'Anger', 'score': 0.029005680233240128},
  {'label': 'Neutral', 'score': 0.011028997600078583},
  {'label': 'Fear', 'score': 0.009845992550253868},
  {'label': 'Anticipation', 'score': 0.008348705247044563},
  {'label': 'Trust', 'score': 0.002228666329756379}]]

Definiuję funkcję, która przekształca wyniki z pipeline w odpowiedni sposób:

In [5]:
def transform_results(result, preds_dict):
    out_dict = {}
    for res in result[0]:
        out_dict[res['label']] = round(res['score'])
    for label in preds_dict.keys():            
            preds_dict[label].append(out_dict[label])

Wczytuję dane do oznaczenia:

In [6]:
with open(file_path, encoding='utf-8') as file:
    whole_text = file.read()
    whole_text = whole_text[len('text\n'):]
    texts = whole_text.split('###########################')

demo = texts[:5]
demo

['Przez 12 lat leczyła mnie ze złym rozpoznaniem.\nTeraz jestem leczony na prawidłową chorobę, biorę odpowiednie leki i od razu czuję się lepiej w porównaniu z tym co było.\nPoza tym każda wizyta, nawet po 10 latach leczenia, wyglądała u niej jak pierwsza tzn nie pamiętała na co choruję, co mi dolega, jakie tabletki biorę.\nNa dodatek pani doktor nie raczyła poinformować mnie też, że leki, które kazała mi brać, powodują uszkodzenia i zniekształcenia płodu! (na szczęście moja dziewczyna nie zaszła w ciążę)\n',
 '\nOSTRZEGAM!!\nOrganizowałam w hotelu imprezę urodzinową na 40 osób.\nPoniewaz impreza była tematyczna, w takim samym klimacie chciałam aby została poprowadzona.\nDj którego zapewnił organizator bladego pojęcia nie miał jaka imprezę będzie prowadził, był kompletnie nieprzygotowany, trzykrotnie mylił imię jubilatki, a wszelkie wcześniejsze ustalenia i zapewnienia menadżera okazały się pustymi słowami.\n"Zaledwie po 3 godzinach ""zabawy"" usłyszałam, ze napoi zimnych nie otrzymamy

Dostałam jeden błąd parę razy kiedy próbowałam przypisać etykiety za pomocą mojego modelu, poniżej jest najlepszy sposób jaki udało mi się wymyślić na radzenie sobie z tym błędem. 

In [7]:
import re

def runtime_handle(error_message):
    # for handling the errors of type 'The size of tensor a (760) must match the size of tensor b (514) at non-singleton dimension 1'
    # which I think are thrown when the input is too long for the model to process
    tensors = list(re.finditer('\(\d+\)', error_message))
    a = int(tensors[0].group()[1:-1])
    b = int(tensors[1].group()[1:-1])
    return (b*0.8)/a

Przypisuję etykiety wszystkim zdaniom i całym opiniom:

In [8]:
from tqdm import tqdm

out_table = pd.read_csv('2024-emotion-recognition\out-header.tsv', sep='\t')
preds_dict = out_table.to_dict(orient='list')

for rev in tqdm(texts):
    sents = rev.split('\n')
    for sent in sents:
        transform_results(nlp(sent), preds_dict)
    try:
        transform_results(nlp(rev), preds_dict)
    except RuntimeError as r:
        frac = runtime_handle(r.args[0])
        transform_results(nlp(rev[:round(len(rev)*frac)]), preds_dict)
    
out_table = pd.DataFrame(preds_dict).astype('bool')

100%|██████████| 168/168 [00:27<00:00,  6.10it/s]


Zapisuję do pliku:

In [9]:
out_table.to_csv('test_B.tsv', sep='\t', index=False)